In [61]:
import json
import pandas as pd
import os
import networkx as nx
import numpy as np
import subprocess
from collections import defaultdict
from collections import Counter
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

# Hyperparameters
alpha1 = 0
alpha2 = 1
beta1 = 1
beta2 = 2
gamma1 = 0
gamma2 = 2
gamma3 = 1

max_epochs = 1000
k = 3

# Files
ori_data = f'dataset/processed_{k}-core_80_20_all.csv'
toy_data = f'dataset/toy_{k}-core_80_20.csv'
ori_label_data = f'dataset/user_label_all.csv'
toy_label_data = f'dataset/toy_user_label.csv'

benign = []
fraudulent = []
total_users = 0

with open(ori_label_data, 'r') as f:
    f.readline()
    for l in f:
        total_users += 1
        s = l.rstrip('\n').split(',')
        if s[2] == 'Benign':
            benign.append(s[0])
        else:
            fraudulent.append(s[0])
print(total_users)

# Convert to networkx
df = pd.read_csv(ori_data)
df['reviewerID'] = 'u' + df['reviewerID'].astype(str)
df['asin'] = 'p' + df['asin'].astype(str)
X = df[['reviewerID', 'asin', 'rating']]
print(X)
G = nx.from_pandas_edgelist(X, source = 'reviewerID', target = 'asin', edge_attr=True, create_using=nx.DiGraph())

nodes = G.nodes()
edges = G.edges(data=True)
print(f"Amazon Network has {len(nodes)} nodes and {len(edges)} edges")

623782
              reviewerID         asin  rating
0        uA27KQ8MMLNFD1B  pB003Y5H5H4     0.0
1         uALHKJO0ZB0WXV  pB000RP4LDU     1.0
2         uAMUNMW13QBFVS  p1463539657     0.5
3        uA273VM3VWW4KNW  pB00KNQLU14     1.0
4        uA33TRNCQK4IUO7  pB000HCPSR6     1.0
...                  ...          ...     ...
5164525  uA1IPQ7J364RU4M  pB006OQ4316     1.0
5164526  uA3MRHSJ3AX9GUT  pB000QSOP0M     0.5
5164527   uA26VFJKQPL67F  pB000H0RZD8     1.0
5164528  uA374ROQPD6L3J3  pB001AVREHA     1.0
5164529  uA2PJUHNKHBDY27  pB002LVU4SE     1.0

[5164530 rows x 3 columns]
Amazon Network has 1245270 nodes and 5164530 edges


In [62]:
user_names = [node for node in nodes if node.startswith('u')]
product_names = [node for node in nodes if node.startswith('p')]
num_users = len(user_names)
num_products = len(product_names)
user_map = dict(zip(user_names, range(len(user_names))))
product_map = dict(zip(product_names, range(len(product_names))))

# Give initial values to each node, product and edge
for node in nodes:
      if node.startswith("u"):
          G.nodes[node]["fairness"] = 1
      else:
          G.nodes[node]["goodness"] = 1

for edge in edges:
    G[edge[0]][edge[1]]["fairness"] = 1

In [63]:
# Update product goodness scores
def Updating_goodness_of_product():
    print('Updating goodness of product')

    def calc_Gp(node):
        inedges = G.in_edges(node, data=True)

        ftotal = len(inedges)
        gtotal = np.sum([edge[2]["fairness"] * edge[2]["rating"]
                          for edge in inedges])

        if ftotal > 0.0:
            mean_rating_fairness = (
                gtotal + beta1*median_gvals) / (ftotal + beta1)
        else:
            mean_rating_fairness = 0.0

        mean_rating_fairness = np.clip(
            mean_rating_fairness, -1, 1)

        return mean_rating_fairness

    # goodness of the curretn products
    currentgvals = [G.nodes[node]["goodness"]
                    for node in nodes if 'p' == node[0]]
    # median
    median_gvals = np.median(currentgvals)

    dp = 0
    for node in nodes:
        if "p" not in node[0]:
            continue

        x = calc_Gp(node)
        dp += abs(G.nodes[node]["goodness"] - x)
        G.nodes[node]["goodness"] = x

    return dp

In [64]:
# Update rating fairness scores
def Updating_fairness_of_ratings():
    print("Updating fairness of ratings")

    def calc_FR(edge):
        user_fairness = G.nodes[edge[0]]["fairness"]  # F(u)

        rating_distance = 1 - \
            (abs(edge[2]["rating"] - G.nodes[edge[1]]["goodness"]) / 2.0)

        x = (gamma1 * user_fairness + gamma2 *
              rating_distance) / (gamma1 + gamma2)
        x = np.clip(x, 0, 1)
        return x

    dr = 0

    for edge in edges:
        x = calc_FR(edge)
        dr += abs(edge[2]["fairness"] - x)
        G.adj[edge[0]][edge[1]]["fairness"] = x
    return dr

In [65]:
# Update user fairness scores
def Updating_fairness_of_users():
    print('updating fairness of users')

    def calc_FU(node):
        outedges = G.out_edges(node, data=True)
        rating_fairness_sum = np.sum(
            [edge[2]["fairness"] for edge in outedges])

        x = (rating_fairness_sum + alpha1*median_fvals) / \
            (len(outedges) + alpha1)
        x = np.clip(x, 0, 1)
        return x

    currentfvals = [G.nodes[node]["fairness"]
                    for node in nodes if 'u' == node[0]]
    # Alternatively, we can use mean here, intead of median
    median_fvals = np.median(currentfvals)

    du = 0

    for node in nodes:
        if "u" not in node[0]:
            continue

        x = calc_FU(node)
        du += abs(G.nodes[node]["fairness"] - x)
        G.nodes[node]["fairness"] = x

    return du

In [66]:
# REV2 algorithm
# Iterate to update parameters
du = 0
dp = 0
dr = 0

for epoch in range(max_epochs):
    print('-----------------')
    print("Epoch number %d with du = %f, dp = %f, dr = %f, for (%d,%d,%d,%d,%d,%d,%d)" % (
        epoch, du, dp, dr, alpha1, alpha2, beta1, beta2, gamma1, gamma2, gamma3))
    if np.isnan(du) or np.isnan(dp) or np.isnan(dr):
        break

    dp = Updating_goodness_of_product()
    dr = Updating_fairness_of_ratings()
    du = Updating_fairness_of_users()

    if du < 0.01 and dp < 0.01 and dr < 0.01:
        break

-----------------
Epoch number 0 with du = 0.000000, dp = 0.000000, dr = 0.000000, for (0,1,1,2,0,2,1)
Updating goodness of product
Updating fairness of ratings
updating fairness of users
-----------------
Epoch number 1 with du = 118936.358739, dp = 194520.614317, dr = 913665.292006, for (0,1,1,2,0,2,1)
Updating goodness of product
Updating fairness of ratings
updating fairness of users
-----------------
Epoch number 2 with du = 11006.627104, dp = 45560.809249, dr = 166501.174439, for (0,1,1,2,0,2,1)
Updating goodness of product
Updating fairness of ratings
updating fairness of users
-----------------
Epoch number 3 with du = 4999.126776, dp = 21095.672607, dr = 69745.858029, for (0,1,1,2,0,2,1)
Updating goodness of product
Updating fairness of ratings
updating fairness of users
-----------------
Epoch number 4 with du = 2010.248962, dp = 8252.156341, dr = 26626.300903, for (0,1,1,2,0,2,1)
Updating goodness of product
Updating fairness of ratings
updating fairness of users
-----------

In [67]:
# Make a directory to save results
Path = './rev2_results/'
if not os.path.exists(Path):
    os.mkdir(Path)
os.listdir('./')

['kcore_5.json.gz',
 '3_Proposed_Methods.ipynb',
 '2_Review_Embedding.ipynb',
 'dataset',
 '1_Dataset_preprocessing_1.ipynb',
 '1_Dataset_preprocessing.ipynb',
 'Baseline.ipynb',
 'rev2_results']

In [68]:
# Make tables to see the score of each user and product
currentfvals = []
for node in nodes:
  if "u" not in node[0]:
    continue
  currentfvals.append(G.nodes[node]["fairness"])
median_fvals = np.median(currentfvals)

user_fairness_df = pd.DataFrame([(x[0], x[1]['fairness']) for x in G.nodes(
                data=True) if x[0].startswith('u')], columns=['source', 'fairness'])
product_goodness_df = pd.DataFrame([(x[0], x[1]['goodness']) for x in G.nodes(data=True) if x[0].startswith('p')],
                    columns=['target', 'goodness'])
edge_fairness_df = nx.to_pandas_edgelist(G)

In [69]:
all_node_vals = []
fair_node_vals = []

for node in nodes:
    if "u" not in node[0]:
        continue
    f = G.nodes[node]["fairness"]
    all_node_vals.append(
        [node, (f - median_fvals) * np.log(G.out_degree(node) + 1), f, G.out_degree(node)])
    fair_node_vals.append(
        [node[1:], (f - median_fvals) * np.log(G.out_degree(node) + 1)])

all_node_vals_sorted = sorted(all_node_vals, key = lambda x: (float(x[1]), float(x[2]), -1 * float(x[3])), reverse=True)
fair_node_vals_sorted = sorted(fair_node_vals, key = lambda x: (float(x[1])), reverse=True)

In [70]:
# Write the score results to two separate csv files
fw = open(Path + f"fng-sorted-users-{alpha1}-{alpha2}-{beta1}-{beta2}-{gamma1}-{gamma2}-{gamma3}.csv", "w")

for i, sl in enumerate(all_node_vals_sorted):
    # if sl[3] in badusers or sl[3] in goodusers:  # dont store users for which we dont have ground truth
    fw.write(f"{str(sl[0])},{str(sl[1])},{str(sl[2])},{str(sl[3])}\n")
fw.close()

fw = open(Path + f"only_fairness-sorted-users-{alpha1}-{alpha2}-{beta1}-{beta2}-{gamma1}-{gamma2}-{gamma3}.csv", "w")

for i, sl in enumerate(fair_node_vals_sorted):
    # if sl[3] in badusers or sl[3] in goodusers:  # dont store users for which we dont have ground truth
    fw.write(f"{str(sl[0])},{str(sl[1])}\n")
fw.close()

os.listdir('./rev2_results/')
print("Number of ground truth bad users = %d, good users = %d" % (len(fraudulent), len(benign)))

Number of ground truth bad users = 22752, good users = 601030


In [71]:
# Calculate the average precision score for fraudulent and benign user prediction (Unsupervised)

fname = "./rev2_results/fng-sorted-users-%d-%d-%d-%d-%d-%d-%d.csv" % (alpha1, alpha2, beta1, beta2, gamma1, gamma2, gamma3)
TOTAL = 100
bashCommand = 'wc -l %s' % fname
NLINES = total_users
bottom_precs = []
top_precs = []
X = []

for K in range(1, TOTAL):
    i = -1
    f = open(fname, "r")
    
    c11, c12, c21, c22 = 0, 0, 0, 0
    x = 0
    for l in f:
        i += 1
        l = l.strip().split(',')
        l[0] = l[0][1:]
        if i < K:
            if l[0] in benign:
                c11 += 1
            elif l[0] in fraudulent:
                c12 += 1
        elif i >= NLINES - K:
            x += 1
            if l[0] in benign:
                c21 += 1
            elif l[0] in fraudulent:
                c22 += 1
    f.close()
    X.append(c21+c22+1)
    bottom_precs.append((c22+0.001)*1.0/(c21+c22+0.001))
    top_precs.append((c11+0.001)*1.0/(c11+c12+0.001))
   
print("Mean average precision:\nFor fraudulent user prediction = %f\nFor benign user prediction = %f" % (np.mean(bottom_precs), np.mean(top_precs)))

Mean average precision:
For fraudulent user prediction = 0.957414
For benign user prediction = 1.000000


In [72]:
# Calculate the AUC score for fraudulent user prediction (Supervised)

scores = defaultdict(list)
f = open("./rev2_results/fng-sorted-users-%d-%d-%d-%d-%d-%d-%d.csv" % (alpha1, alpha2, beta1, beta2, gamma1, gamma2, gamma3), "r")
for l in f:
    l = l.strip().split(",")
    if l[1] == "nan": l[1] = "0"
    scores[l[0]].append(float(l[1]))
    if l[2] == "nan": l[2] = "0"
    scores[l[0]].append(float(l[2]))

X = []
Y = []
for b in benign:
    b_name = 'u' + b
    if len(scores[b_name]) > 0:
        Y.append(0)
        X.append(scores[b_name])
for f in fraudulent:
    f_name = 'u' + f
    if len(scores[f_name]) > 0:
        Y.append(1)
        X.append(scores[f_name])

X = np.array(X)
Y = np.array(Y)
X, Y = shuffle(X, Y)
skf = StratifiedKFold(n_splits=10)
scores = []
aucscores = []
for train, test in skf.split(X, Y):
    train_X = X[train]
    train_Y = Y[train]
    test_X = X[test]
    test_Y = Y[test]

    clf = RandomForestClassifier(n_estimators=500)
    clf.fit(train_X, train_Y)
    scores.append(accuracy_score(test_Y, clf.predict(test_X)))
    try:
        pred_Y = clf.predict_proba(test_X)
        false_positive_rate, true_positive_rate, th =  roc_curve(test_Y, pred_Y[:,1])
        aucscores.append(auc(false_positive_rate, true_positive_rate))
    except:
        pass
    print(scores[-1], aucscores[-1])

print("Accuracy scores", scores, np.mean(scores))
print("AUC scores", aucscores, np.mean(aucscores))

0.9379438593116273 0.6260760708762202
0.9389057214767791 0.6310341931471068
0.9395940876591106 0.6424535719176586
0.9384077719708872 0.6254643448161241
0.9380390522299529 0.6340938604845565
0.9389528359357466 0.6307012083469165
0.9380069896437847 0.6295971878312194
0.9394658373144378 0.6236077298074203
0.9383757093847189 0.6318393168650227
0.9381352399884575 0.6244246132052066
Accuracy scores [0.9379438593116273, 0.9389057214767791, 0.9395940876591106, 0.9384077719708872, 0.9380390522299529, 0.9389528359357466, 0.9380069896437847, 0.9394658373144378, 0.9383757093847189, 0.9381352399884575] 0.9385827104915503
AUC scores [0.6260760708762202, 0.6310341931471068, 0.6424535719176586, 0.6254643448161241, 0.6340938604845565, 0.6307012083469165, 0.6295971878312194, 0.6236077298074203, 0.6318393168650227, 0.6244246132052066] 0.6299292097297451
